# Exploring Generation Capacity Extraction

This notebook helps you:
1. See real examples of capacity mentions in documents
2. Understand which document types contain capacity info
3. Test different extraction approaches

In [ ]:
import pandas as pd
import re
from pathlib import Path
from collections import Counter

# Paths
BASE_DIR = Path.cwd().parent
PROCESSED_DIR = BASE_DIR / "data" / "processed"
ANALYSIS_DIR = BASE_DIR / "data" / "analysis"

print(f"Base directory: {BASE_DIR}")

## 1. Load Clean Energy Projects

In [ ]:
# Load projects
projects = pd.read_parquet(ANALYSIS_DIR / "projects_combined.parquet")
clean_projects = projects[projects['project_energy_type'] == 'Clean'].copy()

print(f"Total projects: {len(projects):,}")
print(f"Clean energy projects: {len(clean_projects):,}")
print()
print("Clean energy by dataset source:")
print(clean_projects['dataset_source'].value_counts())

## 2. Define Capacity Search Terms

We'll use a set of terms to quickly filter sentences that *might* contain capacity information.

In [ ]:
# Terms that indicate generation capacity might be mentioned
CAPACITY_TERMS = {
    # Units
    'mw', 'gw', 'kw', 'mwh', 'gwh', 'kwh',
    'megawatt', 'megawatts', 'gigawatt', 'gigawatts', 
    'kilowatt', 'kilowatts',
    # Context words
    'nameplate', 'capacity', 'generate', 'generating', 
    'generation', 'output', 'rated'
}

def has_capacity_terms(text):
    """Fast check using set intersection."""
    if not text or not isinstance(text, str):
        return False
    words = set(re.findall(r'\b\w+\b', text.lower()))
    return bool(words & CAPACITY_TERMS)

def extract_candidate_sentences(text, context_chars=50):
    """Extract sentences that might contain capacity info."""
    if not text or not isinstance(text, str):
        return []
    
    # Split into sentences (roughly)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    candidates = []
    for sent in sentences:
        if has_capacity_terms(sent) and len(sent) > 30:
            candidates.append(sent.strip())
    
    return candidates

## 3. Sample Some EIS Projects (Most Likely to Have Capacity)

EIS documents are for major projects - they should have capacity info.

In [ ]:
# Get clean energy EIS projects (most likely to have capacity info)
eis_clean = clean_projects[clean_projects['dataset_source'] == 'EIS']
print(f"Clean energy EIS projects: {len(eis_clean):,}")
print()
print("Project types in EIS:")
print(eis_clean['project_type'].value_counts().head(10))

In [ ]:
# Load EIS documents metadata
eis_docs = pd.read_parquet(PROCESSED_DIR / "eis" / "documents.parquet")

# Clean project_id if needed
def extract_id(x):
    if isinstance(x, dict):
        return x.get('value', '')
    return x

eis_docs['project_id'] = eis_docs['project_id'].apply(extract_id)

print(f"EIS documents: {len(eis_docs):,}")
print()
print("Document types:")
print(eis_docs['document_type'].value_counts())

## 4. Look at Specific Projects

Let's pick a few solar/wind projects and examine their documents.

In [ ]:
# Find solar projects in EIS
solar_projects = eis_clean[eis_clean['project_type'].str.contains('solar', case=False, na=False)]
wind_projects = eis_clean[eis_clean['project_type'].str.contains('wind', case=False, na=False)]

print(f"Solar EIS projects: {len(solar_projects)}")
print(f"Wind EIS projects: {len(wind_projects)}")

# Show a few
print("\n=== Sample Solar Projects ===")
solar_projects[['project_id', 'project_title', 'project_type']].head(5)

In [ ]:
# Pick one project to examine in detail
if len(solar_projects) > 0:
    sample_project = solar_projects.iloc[0]
    project_id = sample_project['project_id']
    
    print(f"Project: {sample_project['project_title']}")
    print(f"ID: {project_id}")
    print(f"Type: {sample_project['project_type']}")
    print(f"Location: {sample_project.get('project_location', 'N/A')}")
else:
    # Fallback to any EIS clean project
    sample_project = eis_clean.iloc[0]
    project_id = sample_project['project_id']
    print(f"Using project: {sample_project['project_title']}")

In [ ]:
# Get documents for this project
project_docs = eis_docs[eis_docs['project_id'] == project_id]
print(f"Documents for this project: {len(project_docs)}")
print()
project_docs[['document_id', 'document_type', 'document_title', 'main_document', 'total_pages']].head(10)

## 5. Read Pages and Find Capacity Mentions

Now let's load pages for this project's main document and search for capacity.

In [ ]:
# Get the main document (preferably FEIS or DEIS)
main_docs = project_docs[project_docs['main_document'] == 'YES']
if len(main_docs) == 0:
    main_docs = project_docs

# Prioritize FEIS > DEIS > EA
type_priority = {'FEIS': 1, 'DEIS': 2, 'EA': 3, 'FONSI': 4, '': 5, 'OTHER': 6}
main_docs = main_docs.copy()
main_docs['_priority'] = main_docs['document_type'].map(lambda x: type_priority.get(x, 10))
main_docs = main_docs.sort_values('_priority')

target_doc = main_docs.iloc[0]
target_doc_id = target_doc['document_id']

print(f"Target document: {target_doc['document_title']}")
print(f"Type: {target_doc['document_type']}")
print(f"Pages: {target_doc['total_pages']}")

In [ ]:
# Load pages for this document only (using pyarrow filter)
import pyarrow.parquet as pq

pages_path = PROCESSED_DIR / "eis" / "pages.parquet"

# Read with filter to avoid loading all 6M pages
pages_table = pq.read_table(
    pages_path, 
    filters=[('document_id', '=', target_doc_id)]
)
doc_pages = pages_table.to_pandas()

print(f"Loaded {len(doc_pages)} pages for document {target_doc_id}")

In [ ]:
# Search for capacity mentions in first 30 pages
capacity_findings = []

for _, page in doc_pages.head(30).iterrows():
    page_num = page['page_number']
    text = page['page_text']
    
    candidates = extract_candidate_sentences(text)
    
    for sent in candidates:
        # Look for actual numbers near capacity terms
        if re.search(r'\d+\s*(?:MW|GW|kW|megawatt|gigawatt)', sent, re.IGNORECASE):
            capacity_findings.append({
                'page': page_num,
                'sentence': sent[:300]  # Truncate long sentences
            })

print(f"Found {len(capacity_findings)} sentences mentioning capacity with numbers\n")

for i, finding in enumerate(capacity_findings[:10]):
    print(f"--- Page {finding['page']} ---")
    print(finding['sentence'])
    print()

## 6. Test on Multiple Projects

Let's scan several projects to see the variety of capacity mentions.

In [ ]:
def find_capacity_sentences_for_project(project_id, docs_df, pages_path, max_pages=20):
    """Find capacity-related sentences for a project."""
    # Get project's documents
    project_docs = docs_df[docs_df['project_id'] == project_id]
    if project_docs.empty:
        return []
    
    # Prioritize main documents
    main_docs = project_docs[project_docs['main_document'] == 'YES']
    if main_docs.empty:
        main_docs = project_docs
    
    # Get first main document
    doc_id = main_docs.iloc[0]['document_id']
    
    # Load pages
    try:
        pages_table = pq.read_table(pages_path, filters=[('document_id', '=', doc_id)])
        doc_pages = pages_table.to_pandas()
    except:
        return []
    
    # Search first N pages
    findings = []
    for _, page in doc_pages.head(max_pages).iterrows():
        candidates = extract_candidate_sentences(page['page_text'])
        for sent in candidates:
            if re.search(r'\d+\s*(?:MW|GW|kW|megawatt|gigawatt)', sent, re.IGNORECASE):
                findings.append({
                    'page': page['page_number'],
                    'sentence': sent[:400]
                })
    
    return findings

In [ ]:
# Sample 10 clean energy EIS projects
sample_projects = eis_clean.sample(min(10, len(eis_clean)), random_state=42)

print("Scanning sample projects for capacity mentions...\n")

for _, proj in sample_projects.iterrows():
    findings = find_capacity_sentences_for_project(
        proj['project_id'], 
        eis_docs, 
        pages_path
    )
    
    print(f"=== {proj['project_title'][:60]}... ===")
    print(f"Type: {proj['project_type']}")
    print(f"Found {len(findings)} capacity mentions")
    
    if findings:
        # Show first finding
        print(f"Example (page {findings[0]['page']}): {findings[0]['sentence'][:200]}...")
    print()

## 7. Coverage Analysis

What percentage of projects have capacity mentions in their first 20 pages?

In [ ]:
# Run on a larger sample to estimate coverage
# WARNING: This may take a few minutes

sample_size = 50  # Adjust as needed
sample = eis_clean.sample(min(sample_size, len(eis_clean)), random_state=123)

results = []
for i, (_, proj) in enumerate(sample.iterrows()):
    if i % 10 == 0:
        print(f"Processing {i+1}/{len(sample)}...")
    
    findings = find_capacity_sentences_for_project(
        proj['project_id'], 
        eis_docs, 
        pages_path,
        max_pages=20
    )
    
    results.append({
        'project_id': proj['project_id'],
        'project_title': proj['project_title'],
        'project_type': proj['project_type'],
        'has_capacity_mention': len(findings) > 0,
        'num_mentions': len(findings),
        'first_sentence': findings[0]['sentence'][:200] if findings else None
    })

results_df = pd.DataFrame(results)
print("\n=== Coverage Summary ===")
print(f"Projects with capacity mentions: {results_df['has_capacity_mention'].sum()} / {len(results_df)}")
print(f"Coverage rate: {results_df['has_capacity_mention'].mean()*100:.1f}%")

In [ ]:
# Show projects WITHOUT capacity mentions - these need investigation
no_capacity = results_df[~results_df['has_capacity_mention']]
print(f"\nProjects without capacity mentions ({len(no_capacity)}):")
no_capacity[['project_title', 'project_type']]

## 8. Example: What LLM Input Would Look Like

If we send candidate sentences to an LLM, here's what the input/output would be.

In [ ]:
# Example LLM prompt construction
def build_llm_prompt(sentences, project_title):
    """Build a prompt for LLM extraction."""
    prompt = f"""Extract the proposed project's generation capacity from the following text excerpts.

Project: {project_title}

Return JSON with this structure:
{{
    "capacity_value": <number or null>,
    "capacity_unit": "MW" | "GW" | "kW" | null,
    "confidence": "high" | "medium" | "low",
    "source_quote": "<exact quote from text>"
}}

Rules:
- Extract the PRIMARY project capacity, not alternatives or comparisons
- If multiple values exist, prefer "nameplate" or "rated" capacity
- If text mentions a range (e.g., "50-100 MW"), use the higher value
- Return null values if no clear project capacity is stated

Text excerpts:
"""
    for i, sent in enumerate(sentences[:5]):  # Limit to 5 sentences
        prompt += f"\n[{i+1}] {sent}\n"
    
    return prompt

# Show example
if len(capacity_findings) > 0:
    example_sentences = [f['sentence'] for f in capacity_findings[:5]]
    example_prompt = build_llm_prompt(example_sentences, sample_project['project_title'])
    print("=== Example LLM Prompt ===")
    print(example_prompt)

## 9. Compare EA vs EIS vs CE Coverage

Let's see which document types have the most capacity mentions.

In [ ]:
# Quick check on EA documents
ea_clean = clean_projects[clean_projects['dataset_source'] == 'EA']
ea_docs = pd.read_parquet(PROCESSED_DIR / "ea" / "documents.parquet")
ea_docs['project_id'] = ea_docs['project_id'].apply(extract_id)

print(f"Clean energy EA projects: {len(ea_clean)}")

# Sample and check
ea_sample = ea_clean.sample(min(20, len(ea_clean)), random_state=456)
ea_pages_path = PROCESSED_DIR / "ea" / "pages.parquet"

ea_results = []
for _, proj in ea_sample.iterrows():
    findings = find_capacity_sentences_for_project(
        proj['project_id'], 
        ea_docs, 
        ea_pages_path,
        max_pages=20
    )
    ea_results.append({'has_capacity': len(findings) > 0})

ea_coverage = sum(r['has_capacity'] for r in ea_results) / len(ea_results) * 100
print(f"EA coverage estimate: {ea_coverage:.1f}%")

In [ ]:
# Quick check on CE documents (expect lower coverage)
ce_clean = clean_projects[clean_projects['dataset_source'] == 'CE']
ce_docs = pd.read_parquet(PROCESSED_DIR / "ce" / "documents.parquet")
ce_docs['project_id'] = ce_docs['project_id'].apply(extract_id)

print(f"Clean energy CE projects: {len(ce_clean)}")

# Sample and check
ce_sample = ce_clean.sample(min(20, len(ce_clean)), random_state=789)
ce_pages_path = PROCESSED_DIR / "ce" / "pages.parquet"

ce_results = []
for _, proj in ce_sample.iterrows():
    findings = find_capacity_sentences_for_project(
        proj['project_id'], 
        ce_docs, 
        ce_pages_path,
        max_pages=20
    )
    ce_results.append({'has_capacity': len(findings) > 0})

ce_coverage = sum(r['has_capacity'] for r in ce_results) / len(ce_results) * 100
print(f"CE coverage estimate: {ce_coverage:.1f}%")

## 10. Summary Statistics

In [ ]:
# Compile coverage estimates
coverage_summary = pd.DataFrame([
    {'source': 'EIS', 'clean_projects': len(eis_clean), 'estimated_coverage': f"{results_df['has_capacity_mention'].mean()*100:.1f}%" if len(results_df) > 0 else 'N/A'},
    {'source': 'EA', 'clean_projects': len(ea_clean), 'estimated_coverage': f"{ea_coverage:.1f}%" if 'ea_coverage' in dir() else 'N/A'},
    {'source': 'CE', 'clean_projects': len(ce_clean), 'estimated_coverage': f"{ce_coverage:.1f}%" if 'ce_coverage' in dir() else 'N/A'},
])

print("=== Coverage Summary by Document Source ===")
print(coverage_summary.to_string(index=False))
print()
print("Note: Coverage = % of projects with MW/GW/kW mentions in first 20 pages")

## Next Steps

Based on this exploration:

1. **Prioritize EIS and EA documents** - they have higher coverage rates
2. **Use sentence filtering** - the set-based approach finds relevant text quickly
3. **Consider LLM for extraction** - the filtered sentences are good input for structured extraction
4. **Focus on first 20 pages** - capacity is usually mentioned early in project descriptions